In [1]:
# Import Libraries
import pymongo
import json
from bson.objectid import ObjectId
from datetime import datetime

import time
import mysql.connector as cnx

In [2]:
mydb = cnx.connect(
  host="localhost",
  user="root",
  password="root@123",
  database="mydatabase"
)

mycursor = mydb.cursor(buffered=True)

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/") 
db = client["Tweets_DB"]
tweets_collec = db["Tweets_data"]

In [4]:
# check if the search term starts with '@'
def UserSearch(search_term):
    
    if search_term.startswith('@'):
    # remove the '@' symbol from the search term
        search_term = search_term[1:]
    
    # execute the query to search for user details based on username
        query = """
            SELECT * FROM users 
            WHERE name LIKE %s 
            ORDER BY followers_count DESC, tweets_count DESC, verified DESC
            LIMIT 5
            """
        mycursor.execute(query, ('%' + search_term + '%',))
        results = mycursor.fetchall()
        return results

In [5]:
def get_user_tweets(user_id):
    user_tweets = list(tweets_collec.find({'User_Id': user_id}).sort([('created_at', -1)]).limit(3))
    tweet_details = []
    for tweet in user_tweets:
        tweet_details.append({
            'created_at': tweet['created_at'],
            'text': tweet['Text'],
            'hashtags': tweet['Hashtag'],
            'retweet_count': tweet['Retweet_Count'],
            'likes_count': tweet['Likes_Count']
        })
    return tweet_details

In [6]:
tweets_cache={}
def UserPrint(results):
    for result in results:
        user_id = result[0]
        tweets_cache[user_id] = get_user_tweets(user_id)
        if result[3]==1:
            verified_status="✅"
        else:
            verified_status="❌"
            
        line1 = "ID: {} | Name: {} | Verified: {}".format(result[0], result[1], verified_status)
        # format the remaining fields in another line
        line2 = "Followers: {} | Tweets: {}".format(result[4], result[8])
        line3 = "Description : {}".format(result[9])
        line4="Location : {} | Creation Date:{}".format(result[7],result[6])
        
        # print both lines
        print(line1)
        print(line2)
        print(line3)
        print(line4)
        print("--------------------------------------------")

In [10]:
search_term = input("Enter the search term: ")
results=UserSearch(search_term)
UserPrint(results[:3])
    # check if there are more results
if len(results) > 3:
        # prompt the user to load more results
    load_more = input("Load more results? (yes/no) ")
    if load_more.lower().startswith('y'):
        UserPrint(results[3:5])
user_choice = int(input("Enter the number of the user whose tweets you want to see: "))

# Get the user_id of the selected user
user_id = results[user_choice-1][0]

# Display the tweets of the selected user

if user_id in tweets_cache:
    print(f"Tweets of {results[user_choice-1][1]}:")
    for tweet in tweets_cache[user_id]:
        print(tweet)
else:
    print("No tweets found for the selected user.")

Enter the search term: @sai
ID: 56304605 | Name: Rajdeep Sardesai | Verified: ✅
Followers: 8947331 | Tweets: 1
Description : Citizen first. Only 'ism' is humanism. newsman, tv anchor, author, father, friend. New book: 2019: How Modi Won India. pre order here: http://bit.ly/HowModiWon
Location : New Delhi | Creation Date:2009-07-13 06:14:44
--------------------------------------------
ID: 260114837 | Name: Saint Laurent Don | Verified: ✅
Followers: 3691645 | Tweets: 1
Description : SavageMode
Location : Atlanta, GA | Creation Date:2011-03-03 06:42:26
--------------------------------------------
ID: 286042531 | Name: Syed Talat Hussain | Verified: ✅
Followers: 3348942 | Tweets: 1
Description : Journalist, writer, obsessive jogger, voracious reader, massive meat-eater
Location : Islamabad | Creation Date:2011-04-22 07:21:57
--------------------------------------------
Load more results? (yes/no) no
Enter the number of the user whose tweets you want to see: 1
Tweets of Rajdeep Sardesai:
{'